In [ ]:
# -*- coding: utf-8 -*-
#!/usr/bin/python3
### Multiple threads
### 
import _thread
import time

def print_time(threadName,delay):
    count = 0
    while count < 5:
        time.sleep(delay)
        count += 1
        print("%s: %s" % (threadName,time.ctime(time.time())))
        
try:
    _thread.start_new_thread(print_time,("Thread-1",2,)) # 两个是同时进行了，两个人在同一张桌子上吃饭
    print("---------------------------------------------")
    _thread.start_new_thread(print_time,("Thread-2",4,))
except:
    print("Error:无法启动线程")
    
while 1:
    pass

In [4]:
import threading
import time

exitFlag = 0

# 继承了threading.Thread的所有模块
class myThread(threading.Thread):
    def __init__(self,threadID,name,counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
    
    def run(self):
        print("开始线程" + self.name)
        print_time(self.name,self.counter,5)
        print("退出线程" + self.name)
    
def print_time(threadName,delay,counter):
    while counter:
        if exitFlag:
            threadName.exit()
        time.sleep(delay)
        print("%s: %s" % (threadName,time.ctime(time.time())))
        counter -= 1

thread1 = myThread(1,"Thread-1",1) # 最后一个参数是delay
thread2 = myThread(2,"Thread-2",2)

thread1.start()
thread2.start()
thread1.join()
thread2.join()
print("退出主线程")

开始线程Thread-2开始线程Thread-1

Thread-1: Sun Jan 28 10:43:34 2018
Thread-1: Sun Jan 28 10:43:35 2018
Thread-2: Sun Jan 28 10:43:35 2018
Thread-1: Sun Jan 28 10:43:36 2018
Thread-1: Sun Jan 28 10:43:37 2018
Thread-2: Sun Jan 28 10:43:37 2018
Thread-1: Sun Jan 28 10:43:38 2018
退出线程Thread-1
Thread-2: Sun Jan 28 10:43:39 2018
Thread-2: Sun Jan 28 10:43:41 2018
Thread-2: Sun Jan 28 10:43:43 2018退出主线程

退出线程Thread-2


In [8]:
import threading
import time 

### 有的时候要同步阻塞
### 这相当于暂时单线程
class myThread(threading.Thread):
    def __init__(self,threadID,name,counter):
        threading.Thread.__init__(self)
        self.threadID = threadID 
        self.name = name
        self.counter = counter
    
    def run(self):
        print("开启线程" + self.name)
        ### 获取锁，用于线程同步（阻塞同步的进程）
        threadLock.acquire()
        print_time(self.name,self.counter,3)
        ### 释放锁
        threadLock.release()
        
def print_time(threadName,delay,counter):
    while counter:
        time.sleep(delay)
        print("%s: %s" % (threadName,time.ctime(time.time())))
        counter -= 1
    
threadLock = threading.Lock()
threads = []

thread1 = myThread(1,"Thread-1",1)
thread2 = myThread(2,"Thread-2",2)

thread1.start()
thread2.start()

threads.append(thread1)
threads.append(thread2)

for t in threads:
    t.join()
print("退出主线程")

开启线程Thread-1
开启线程Thread-2
Thread-1: Sun Jan 28 10:56:34 2018
Thread-1: Sun Jan 28 10:56:35 2018
Thread-1: Sun Jan 28 10:56:36 2018
Thread-2: Sun Jan 28 10:56:38 2018
Thread-2: Sun Jan 28 10:56:40 2018
Thread-2: Sun Jan 28 10:56:42 2018退出主线程



In [10]:
import queue
import threading 
import time

exitFlag = 0
class myThread(threading.Thread):
    def __init__(self,threadID,name,q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
        
    def run(self):
        print("开启线程" + self.name)
        process_data(self.name,self.q)
        print("退出线程" + self.name)
        
def process_data(threadName,q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            data = q.get()
            queueLock.release()
            print("%s processing %s" % (threadName,data))
        else:
            queueLock.release()
        time.sleep(1)
        
threadList = ["Thread-1","Thread-2","Thread-3"]
nameList = ["One","Two","Three","Four","Five"]
queueLock = threading.Lock()
workQueue = queue.Queue(10)
threads = []
threadID = 1

for tName in threadList:
    thread = myThread(threadID,tName,workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1
    
queueLock.acquire()
for word in nameList:
    workQueue.put(word)
queueLock.release()

while not workQueue.empty():
    pass

exitFlag = 1

for t in threads:
    t.join()
print("退出主线程")

    
            

开启线程Thread-1
开启线程Thread-2开启线程Thread-3

Thread-3 processing One
Thread-1 processing TwoThread-2 processing Three

Thread-3 processing Four
Thread-2 processing Five
退出线程Thread-3
退出线程Thread-1退出线程Thread-2退出主线程


